In [9]:
from datasets import load_dataset

ds = load_dataset("Shengtao/recipe")

print(len(ds['train']))

32722


In [26]:
import openai

# api
openai.api_key = ""

def call_chatgpt_api(user_input, model="gpt-3.5-turbo", system_message="You are a professional chef"):
    response = openai.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": user_input}
            ]
        )
    return response.choices[0].message.content

Load BERT to calculate similarity score

In [27]:
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
import torch


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def get_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze()

def cosine_similarity_score(text1, text2):
    emb1 = get_embedding(text1).numpy().reshape(1, -1)
    emb2 = get_embedding(text2).numpy().reshape(1, -1)
    return cosine_similarity(emb1, emb2)[0][0]

In [29]:
titles = ds['train']['title']
ingredients = ds['train']['ingredients']

for i in range(2):
    title = titles[i]
    query = f"What's the ingredients of {title}? Please list the specific amount of each ingredient only, separate with semicolons."
    print(f"Query: {query}")
    
    ## TODO: link to the chef de cuisine
    response = call_chatgpt_api(query)
    print(f"Generated response: {response}")
    
    ingredient = ingredients[i]
    print(f"Standard ingredients: {ingredient}")
    
    similarity_score = cosine_similarity_score(response, ingredient)
    print(f"Cosine similarity: {similarity_score:.4f}\n")

Query: What's the ingredients of Simple Macaroni and Cheese? Please list the specific amount of each ingredient only, separate with semicolons.
Generated response: 8 oz macaroni; 2 cups shredded cheddar cheese; 2 tbsp butter; 2 tbsp all-purpose flour; 2 cups milk; Salt and pepper to taste
Standard ingredients: 1 (8 ounce) box elbow macaroni ; ¼ cup butter ; ¼ cup all-purpose flour ; ½ teaspoon salt ;   ground black pepper to taste ; 2 cups milk ; 2 cups shredded Cheddar cheese
Cosine similarity: 0.9573

Query: What's the ingredients of Gourmet Mushroom Risotto? Please list the specific amount of each ingredient only, separate with semicolons.
Generated response: Arborio rice; 1 cup
Assorted gourmet mushrooms (such as shiitake, oyster, and cremini); 1 1/2 cups sliced
Shallots; 2, finely chopped
Garlic cloves; 2, minced
Dry white wine; 1/2 cup
Vegetable or chicken broth; 4 cups
Butter; 2 tablespoons
Olive oil; 2 tablespoons
Parmesan cheese; 1/2 cup, grated
Salt; 1 teaspoon
Black pepper; 